# Redes Neuronales - Trabajo Práctico N° 1 - Ejercicio 2 - Notebook #2
En esta segunda notebook, se busca definir cuál métrica es más apropiada para analizar la performance del modelo y qué hiper parámetros se van a utilizar para el ajuste del modelo acorde a la validación. Finalmente, estas decisiones se vuelcan en la selección del mejor modelo para el problema de la clasificación de correos electrónicos asociados grupos de noticias.

### Integrantes del grupo
* Gaytan, Joaquín Oscar
* Kammann, Lucas Agustín

# 1. Métrica
La métrica a utilizar para cuantificar la performance de los modelos, seleccionar los hiperparámetros y validarlos, será la **sensibilidad** o **recall**.

## 1.1. Justificación

# 2. Preparación de los datasets

## 2.1. Cargando el dataset original

In [40]:
import pandas as pd

In [41]:
# Read database from .csv
df = pd.read_csv('../assets/diabetes.csv', delimiter=',')

In [42]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## 2.2. Filtrado de valores inválidos
Se filtran los valores que se consideran inválidos para las variables en cuestión, estas consideraciones se obtuvieron como resultado del análisis realizado en el notebook #1.

In [43]:
import numpy as np

In [44]:
# Filtering Glucose values
df['Glucose'].replace(0, np.nan, inplace=True)

# Filtering Blood Pressure values
df['BloodPressure'].replace(0, np.nan, inplace=True)

# Filtering Skin Thickness values
df['SkinThickness'].replace(0, np.nan, inplace=True)

# Filtering Insulin values
df['Insulin'].replace(0, np.nan, inplace=True)

# Filtering Body Mass Index values
df['BMI'].replace(0, np.nan, inplace=True)

## 2.3. Filtrado de outliers
Por el momento, se consideró no emplear un filtrado de outliers.

## 2.4. Separación de datasets
Se separa el dataset original en los datasets de train, valid y test. Además, se debe corregir que los valores inválidos del dataset original fueron reemplazados por el valor NaN.

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
# Splitting into the total train and the test datasets, because
# the total train contains the train and valid datasets used for
# hiper parameter selection
total_train, test = train_test_split(df, test_size=0.2, random_state=24)

In [47]:
# Creating the datasets for the hiper parameter selection
train, valid = train_test_split(total_train, test_size=0.2, random_state=24)

In [48]:
train_means = train.mean().to_numpy()

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(total_train.columns):
    total_train.loc[:,column].replace(np.nan, train_means[index], inplace=True)

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(train.columns):
    train.loc[:,column].replace(np.nan, train_means[index], inplace=True)

# Replacing nan values of the test dataset with training mean values
for index, column in enumerate(test.columns):
    test.loc[:,column].replace(np.nan, train_means[index], inplace=True)

# Replacing nan values of the valid dataset with training mean values
for index, column in enumerate(valid.columns):
    valid.loc[:,column].replace(np.nan, train_means[index], inplace=True)

C:\anaconda\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [75]:
# Extracting the inputs and outputs of the total train dataset
x_total_train = total_train.to_numpy()[:,:8]
y_total_train = total_train.to_numpy()[:,8]

# Extracting the inputs and outputs of the train dataset
x_train = train.to_numpy()[:,:8]
y_train = train.to_numpy()[:,8]

# Extracting the inputs and outputs of the valid dataset
x_valid = valid.to_numpy()[:,:8]
y_valid = valid.to_numpy()[:,8]

# Extracting the inputs and outputs of the test dataset
x_test = test.to_numpy()[:,:8]
y_test = test.to_numpy()[:,8]

# 3. Selección de modelo e hiper parámetros

In [62]:
from sklearn.metrics import recall_score

## 3.1. Entrenamiento de todos los modelos

In [87]:
%%time

from src.gaussian_naive_bayes import BinaryGaussianNaiveBayes

# Creating the lists with model parameters and their score
model_params = []
model_score = []

# Iterating over all hiper parameters to find the best model
for use_filter in [[(i & (0x1 << (7-j)) > 0) for j in range(8)] for i in range(256)]:
    for use_smoothing in np.linspace(0, 0.1, 100):

        # Create and train the model
        classifier = BinaryGaussianNaiveBayes(smoothing=use_smoothing)
        classifier.fit(x_train[:,use_filter], y_train)

        # Estimate and predict with the model
        predictions = classifier.predict(x_valid[:,use_filter])
        score = recall_score(y_valid, predictions)
        params = {
            'use_smoothing': use_smoothing,
            'use_filter': use_filter
        }

        # Save the results
        model_params.append(params)
        model_score.append(score)

Wall time: 7min 24s


## 3.2. Mejor modelo

In [88]:
# Search the model with the best score
selected_model_index = np.argmax(model_score)
selected_model_score = model_score[selected_model_index]
selected_model_params = model_params[selected_model_index]

In [89]:
selected_model_params

{'use_smoothing': 0.07171717171717172,
 'use_filter': [False, True, True, True, False, True, False, False]}

## 3.3. Entrenamiento completo

In [90]:
# Create and train the model
classifier = BinaryGaussianNaiveBayes(smoothing=selected_model_params['use_smoothing'])
classifier.fit(x_total_train, y_total_train)

# 4. Validación y performance del modelo

In [95]:
predictions = classifier.predict(x_test[:,selected_model_params['use_filter']])

In [96]:
score = recall_score(y_test, predictions)

In [97]:
print(score)

0.7857142857142857
